In [0]:
-- Magic Commands

-- %python
-- %sql
-- %md
-- %md-sandbox
-- %run ./Includes/Setup
-- %fs ls 'databricks-datasets' #
-- files = dbutils.fs.ls('/databricks-datasets')
-- print(files)
-- display(files)
-- dbutils.help() 
-- dbutils.fs.help()

In [0]:
-- USE CATALOG hive_metastore;

-- DESCRIBE DETAIL table_name;
-- DESCRIBE HISTORY table_name;
-- DESCRIBE DATABASE EXTENDED new_default;
-- RESTORE TABLE employees TO VERSION AS OF 5;
-- OPTIMIZE employees ZORDER BY id;
-- VACUUM employees [RETAIN 0 HOURS];

-- %fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'
-- %fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'
-- SET spark.databricks.delta.retentionDurationCheck.enabled = false;

In [0]:
CREATE TABLE employees
  (id INT, name STRING, salary DOUBLE);
  -- USING delta  # default table type

INSERT INTO employees
VALUES 
  (1, "Adam", 3500.0),
  (2, "Sarah", 4020.5);

INSERT INTO employees
VALUES
  (3, "John", 2999.3),
  (4, "Thomas", 4000.3);

INSERT INTO employees
VALUES
  (5, "Anna", 2500.0);

INSERT INTO employees
VALUES
  (6, "Kim", 6200.3);

SELECT * FROM employees;

DESCRIBE DETAIL employees;
-- %fs ls 'dbfs:/user/hive/warehouse/employees'

UPDATE employees 
SET salary = salary + 100
WHERE name LIKE "A%";

DESCRIBE HISTORY employees;
--  %fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'
-- %fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'

In [0]:
-- SELECT * FROM employees VERSION AS OF 4;
-- SELECT * FROM employees@v4
-- DELETE FROM employees;
-- SELECT * FROM employees;
-- RESTORE TABLE employees TO VERSION AS OF 5;
-- SELECT * FROM employees;
-- DESCRIBE HISTORY employees
-- DESCRIBE DETAIL employees; 
-- OPTIMIZE employees ZORDER BY id;
-- DESCRIBE DETAIL employees;
-- DESCRIBE HISTORY employees;


In [0]:
-- %fs ls 'dbfs:/user/hive/warehouse/employees'
-- VACUUM employees;
-- VACUUM employees RETAIN 0 HOURS; -- throws error as the retenstion is 7 by default

-- %fs ls 'dbfs:/user/hive/warehouse/employees'
-- SET spark.databricks.delta.retentionDurationCheck.enabled = false;

-- VACUUM employees RETAIN 0 HOURS
-- SELECT * FROM employees@v1
-- DROP TABLE employees;
-- SELECT * FROM employees
--  %fs ls 'dbfs:/user/hive/warehouse/employees'


In [0]:
USE CATALOG hive_metastore;

-- CREATE TABLE managed_default
--   (width INT, length INT, height INT);

-- INSERT INTO managed_default VALUES (3 INT, 2 INT, 1 INT);

-- DESCRIBE EXTENDED managed_default;

CREATE TABLE external_default
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_default';
  
INSERT INTO external_default
VALUES (3 INT, 2 INT, 1 INT);


-- DROP TABLE managed_default;
-- %fs ls 'dbfs:/user/hive/warehouse/managed_default'

-- DROP TABLE external_default;
-- %fs ls 'dbfs:/mnt/demo/external_default'

In [0]:
-- CREATE SCHEMA new_default;
-- DESCRIBE DATABASE EXTENDED new_default;

-- USE new_default;

-- CREATE TABLE managed_new_default
--   (width INT, length INT, height INT);
  
-- INSERT INTO managed_new_default VALUES (3 INT, 2 INT, 1 INT);

-- CREATE TABLE external_new_default
--   (width INT, length INT, height INT)
-- LOCATION 'dbfs:/mnt/demo/external_new_default';
  
-- INSERT INTO external_new_default VALUES (3 INT, 2 INT, 1 INT);

DESCRIBE EXTENDED managed_new_default;
DESCRIBE EXTENDED external_new_default;

-- SELECT * FROM external_new_default;

-- DROP TABLE managed_new_default;
-- DROP TABLE external_new_default;

-- %fs ls 'dbfs:/user/hive/warehouse/new_default.db/managed_new_default'
-- %fs ls 'dbfs:/mnt/demo/external_new_default'

In [0]:
-- CREATE SCHEMA custom 
-- LOCATION 'dbfs:/Shared/schemas/custom.db';

-- DESCRIBE DATABASE EXTENDED custom;

-- USE custom;

-- CREATE TABLE managed_custom
--   (width INT, length INT, height INT);
  
-- INSERT INTO managed_custom
-- VALUES (3 INT, 2 INT, 1 INT);

-- CREATE TABLE external_custom
--   (width INT, length INT, height INT)
-- LOCATION 'dbfs:/mnt/demo/external_custom';
  
-- INSERT INTO external_custom
-- VALUES (3 INT, 2 INT, 1 INT);

-- DESCRIBE EXTENDED managed_custom;
-- DESCRIBE EXTENDED external_custom;

-- DROP TABLE managed_custom;
-- DROP TABLE external_custom;

-- %fs ls 'dbfs:/Shared/schemas/custom.db/managed_custom'
-- %fs ls 'dbfs:/mnt/demo/external_custom'


In [0]:
-- ### 1. VIEWS  ####

USE CATALOG hive_metastore;

CREATE TABLE IF NOT EXISTS smartphones
(id INT, name STRING, brand STRING, year INT);

INSERT INTO smartphones
VALUES (1, 'iPhone 14', 'Apple', 2022),
      (2, 'iPhone 13', 'Apple', 2021),
      (3, 'iPhone 6', 'Apple', 2014),
      (4, 'iPad Air', 'Apple', 2013),
      (5, 'Galaxy S22', 'Samsung', 2022),
      (6, 'Galaxy Z Fold', 'Samsung', 2022),
      (7, 'Galaxy S9', 'Samsung', 2016),
      (8, '12 Pro', 'Xiaomi', 2022),
      (9, 'Redmi 11T Pro', 'Xiaomi', 2022),
      (10, 'Redmi Note 11', 'Xiaomi', 2021);

-- SHOW TABLES;

CREATE VIEW view_apple_phones
AS  SELECT * 
    FROM smartphones 
    WHERE brand = 'Apple';

SELECT * FROM view_apple_phones;

CREATE TEMP VIEW temp_view_phones_brands
AS  SELECT DISTINCT brand
    FROM smartphones;

SELECT * FROM temp_view_phones_brands;

-- CREATE GLOBAL TEMP VIEW global_temp_view_latest_phones
-- AS SELECT * FROM smartphones
--     WHERE year > 2020
--     ORDER BY year DESC;

SELECT * FROM global_temp.global_temp_view_latest_phones;

-- SHOW TABLES;
SHOW TABLES IN global_temp;

DROP TABLE smartphones;
DROP VIEW view_apple_phones;
DROP VIEW global_temp.global_temp_view_latest_phones;


In [0]:
-- 2. ###   ###